# PROJECT 

### Import and ROS initialization

This section sets up the core components required for integrating ROS with the Jupyter environment. It imports necessary libraries, initializes global variables, and prepares the system for robot control, real-time data handling, and interaction through widgets and plots. This foundational setup enables communication with ROS topics, services, and actions, and is essential for the rest of the notebook to function correctly.

In [ ]:
import rospy
import threading
import time
import actionlib
import sys

from nav_msgs.msg import Odometry
from geometry_msgs.msg import Pose, Twist, PoseStamped
from actionlib_msgs.msg import GoalStatus

from assignment2_1_rt.msg import PosVel, PlanningAction, PlanningGoal

import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

# required global variables
OdPose = Pose()
OdTwist = Twist()
client = None
goal_handle = None
goal_active = False
pub_PsVl = None

### Odom callback and publisher

This section defines the callback functions for handling odometry data and periodically publishing the robot's position and velocity. It ensures the system stays updated with the robot's current state and makes this information available to other ROS components through a custom message.

In [ ]:
def odom_callback(msg):
    global OdPose, OdTwist
    OdPose = msg.pose.pose
    OdTwist = msg.twist.twist

def pos_callback(event=None):
    global OdPose, OdTwist, pub_PsVl
    if pub_PsVl:
        msg = PosVel()
        msg.pos_x = OdPose.position.x
        msg.pos_y = OdPose.position.y
        msg.vel_x = OdTwist.linear.x
        msg.vel_z = OdTwist.angular.z
        pub_PsVl.publish(msg)

### Send and cancel goal functions for goal management

This section defines the functions to send and cancel navigation goals via the action client. When a new goal is sent, the target position is taken from the input widgets and transmitted to the planner. If a goal is active, it can also be canceled, providing interactive control over the robot’s behavior directly from the notebook interface.

In [ ]:
def send_goal(_):
    global client, goal_active
    if not goal_active:
        x = x_input.value
        y = y_input.value

        target = PoseStamped()
        target.pose.position.x = x
        target.pose.position.y = y

        goal = PlanningGoal()
        goal.target_pose = target

        client.send_goal(goal, done_cb=done_cb)
        goal_active = True
        with goal_output:
            clear_output()
            print(f"Sent goal: ({x}, {y})")

def cancel_goal(_):
    global client, goal_active
    if goal_active:
        client.cancel_goal()
        goal_active = False
        with goal_output:
            clear_output()
            print("Goal canceled.")
            
def done_cb(status, result):
    global goal_active
    goal_active = False

    #with goal_output:
     #   clear_output()
      #  if status == GoalStatus.SUCCEEDED:
       #     print(f"[RESULT] - Goal reached! Final position: ({result.final_x:.2f}, {result.final_y:.2f})")
        #else:
         #   print(f"[RESULT] - Goal failed with status: {status}")


### Ros Node initialization function

This section initializes the ROS node and sets up communication with the rest of the ROS ecosystem. It subscribes to odometry data, advertises a publisher for custom position and velocity messages, and connects to the action server used for goal planning. A timer is also started to periodically publish the robot’s state.

In [ ]:
def init_ros_node():
    global client, pub_PsVl
    rospy.init_node('interactive_action_client')
    rospy.Subscriber("/odom", Odometry, odom_callback)
    pub_PsVl = rospy.Publisher("/posVel", PosVel, queue_size=10)
    
    client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
    rospy.loginfo("Waiting for action server...")
    client.wait_for_server()
    rospy.loginfo("Action server connected.")

    rospy.Timer(rospy.Duration(0.5), pos_callback)

init_ros_node()

### Interactive widgets for control purposes

This section creates interactive input fields and buttons using `ipywidgets` to allow the user to send or cancel navigation goals. The X and Y fields define the target position, while the "Send Goal" and "Cancel Goal" buttons trigger the corresponding ROS actions. Output feedback is displayed below the controls.

In [ ]:
x_input = widgets.FloatText(value=0.0, description='X:', step=0.1)
y_input = widgets.FloatText(value=0.0, description='Y:', step=0.1)

send_button = widgets.Button(description='Send Goal', button_style='success')
cancel_button = widgets.Button(description='Canceal Goal', button_style='danger')

goal_output = widgets.Output()

display(widgets.HBox([x_input, y_input]))
display(widgets.HBox([send_button, cancel_button]))
display(goal_output)

Eventually send/cancel goal

In [ ]:
send_button.on_click(send_goal)
cancel_button.on_click(cancel_goal)

### Position live check

This section creates an interactive plot to visualize the robot's position and its movement trajectory over time. The robot's position is updated in real-time, while the trajectory can be optionally enabled or disabled through a dropdown menu. Additionally, the "Reset" button allows for clearing the trace. The plot dynamically updates based on the robot's odometry data, providing an interactive visualization of the robot's movement. Key features:
- **Trace Control**: The dropdown menu allows switching between keeping the trace of the robot's movement or disabling it.
- **Reset Trace**: The "Reset" button clears the current trajectory trace on the plot.
- **Real-time Updates**: The robot's current position is updated every 0.5 seconds and plotted on the graph.

In [ ]:
%matplotlib notebook
fig, ax = plt.subplots()
robot_pos, = ax.plot([], [], 'ro', label='Robot')
trace_line, = ax.plot([], [], 'b-', alpha=0.5, label='Trace')

ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)
ax.grid(True)
ax.legend()

# Global variables used for the trace
trace_enabled = True
trace_visible = True
trace_x = []
trace_y = []

# Dropdown menu
trace_mode_dropdown = widgets.Dropdown(
    options=['Keep trace','Disable trace'],
    value='Keep trace',
    description='Trace:'
)

# Trace-reset button
reset_button = widgets.Button(description='Reset', button_style='info')

# Dropdown menu logic
def on_trace_mode_change(change):
    global trace_enabled, trace_visible
    mode = change['new']
    if mode == 'Keep trace':
        trace_enabled = True
        trace_visible = True
    elif mode == 'Disable trace':
        trace_enabled = False
        trace_visible = False
        trace_line.set_data([], [])
        fig.canvas.draw_idle()

# Reset button logic
def reset_trace(_):
    global trace_x, trace_y
    trace_x.clear()
    trace_y.clear()
    trace_line.set_data([], [])
    clear_output()
    fig.canvas.draw_idle()

# Connect callbacks
trace_mode_dropdown.observe(on_trace_mode_change, names='value')
reset_button.on_click(reset_trace)

# Display widgets
display(widgets.HBox([trace_mode_dropdown, reset_button]))

# Real-time plot updater
def update_plot():
    while not rospy.is_shutdown():
        x = OdPose.position.x
        y = OdPose.position.y
        robot_pos.set_data(x, y)
        ax.set_title(f"Robot Position: x={x:.2f}, y={y:.2f}")

        if trace_enabled:
            trace_x.append(x)
            trace_y.append(y)
        if trace_visible:
            trace_line.set_data(trace_x, trace_y)

        fig.canvas.draw_idle()
        time.sleep(0.5)

# Start update thread
threading.Thread(target=update_plot, daemon=True).start()


### Cleaning

Disabled: useless when using the 'run all option'.

In [ ]:
#def shutdown_ros(): # Node shutdown
 #   rospy.signal_shutdown("Interruzione notebook")